In [2]:
import os
import requests
import re
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import finnhub
from dotenv import load_dotenv
from pathlib import Path    
import numpy as np
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import sys
import time
sys.path.append('../') # Change the python path at runtime

# Self-created modules
from src.utils import path as path_yq



/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up Variables

In [3]:
load_dotenv()

SUBJECTIVITY_API_KEY = os.environ.get('SUBJECTIVITY_API_KEY')
POLARITY_API_KEY = os.environ.get('POLARITY_API_KEY')
INTENSITY_API_KEY = os.environ.get('INTENSITY_API_KEY')
FINNHUB_API_KEY = os.environ.get('FINNHUB_API_KEY')

In [51]:
BACKTEST_START_DATE = '2023-11-01'
BACKTEST_END_DATE = '2024-01-28'

cur_dir = Path.cwd()
root_dir = path_yq.get_root_dir(cur_dir)

# Finnhub API

- Finnhub API
https://finnhub.io/docs/api/company-news

# Fetch News Data

UNIX timezone 1706743226
https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjMxeO_w5CEAxWByjgGHQwwCT8QFnoECA0QAw&url=https%3A%2F%2Fnote.nkmk.me%2Fen%2Fpython-unix-time-datetime%2F%23%3A~%3Atext%3DUnix%2520time%2520is%2520the%2520number%2CPosix%2520time%252C%2520and%2520so%2520on.&usg=AOvVaw0s2rDBPUsnO1N5cO74o2AR&opi=89978449

TODO: Need to remove similar news (might not be the same title)

- 2024-02-04: API down 

In [52]:
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# print(finnhub_client.general_news('general', min_id=0))

# Need to use _from instead of from to avoid conflict

#TODO: Cannot fetch all days together. Need to fetch one by one, but rmb rate limit
news_resp = finnhub_client.company_news('BA', 
                                        _from=BACKTEST_START_DATE, 
                                        to=BACKTEST_END_DATE)

# TODO: Explore
# Company Peers
# print(finnhub_client.company_peers('AAPL'))

In [53]:
pd.DataFrame(news_resp)

,category,datetime,headline,id,image,related,source,summary,url
0,company,1706482141,"Stocks Could Roll Over, Good For Traders, Not ...",125370134,https://static.seekingalpha.com/cdn/s3/uploads...,BA,SeekingAlpha,A sell-off in tech stocks may occur in Februar...,https://finnhub.io/api/news?id=67b909653cc0605...
1,company,1706471533,UPDATE 1-United CEO kickstarts Airbus talks am...,125369404,https://s.yimg.com/cv/apiv2/social/images/yaho...,BA,Yahoo,United Airlines has approached Airbus about bu...,https://finnhub.io/api/news?id=8aa5a96cfa88cfc...
2,company,1706469720,Boeing’s 737 MAX 9 Is Back. Here’s What Comes ...,125370035,https://s.yimg.com/ny/api/res/1.2/94XxjyNpA3x2...,BA,Yahoo,Alaska and United Airlines are both flying the...,https://finnhub.io/api/news?id=15a46b2755dfddc...
3,company,1706468347,United CEO kickstarts Airbus talks amid Boeing...,125369405,https://media.zenfs.com/en/reuters.com/a53fe31...,BA,Yahoo,DUBLIN/CHICAGO (Reuters) -United Airlines has ...,https://finnhub.io/api/news?id=d2cf185b26a7531...
4,company,1706454000,David Calhoun Was Brought In to Fix Boeing. No...,125365059,https://s.yimg.com/ny/api/res/1.2/ZnyMlOlQ4slV...,BA,Yahoo,"Calhoun, named CEO of Boeing in 2020 after a p...",https://finnhub.io/api/news?id=7a391180c3d4b9f...
...,...,...,...,...,...,...,...,...,...
205,company,1706035095,"United CEO Blames Boeing for Max 9 Missteps, W...",125255730,https://s.yimg.com/ny/api/res/1.2/8kLuo_U6PqSb...,BA,Yahoo,(Bloomberg) -- United Airlines Holdings Inc. i...,https://finnhub.io/api/news?id=95e9550b01b84a4...
206,company,1706033820,United pulls plans for Boeing’s biggest 737 Ma...,125266873,https://images.mktw.net/im-71084564/social,BA,MarketWatch,United Airlines Holdings Inc. on Tuesday said ...,https://finnhub.io/api/news?id=ec55b6dcfedfe40...
207,company,1706028000,Boeing Co. stock underperforms Tuesday when co...,125268547,https://images.mktw.net/im-220105/social,BA,MarketWatch,Shares of Boeing Co. slipped 1.60% to $211.50 ...,https://finnhub.io/api/news?id=160971fbb23767d...
208,company,1706025730,Boeing CEO to meet with senators after 737 MAX...,125260513,,BA,Finnhub,Boeing CEO Dave Calhoun willbe on Capitol Hill...,https://finnhub.io/api/news?id=9c68b95d29147e9...


In [55]:
pd.to_datetime(1706023800, unit="s")

Timestamp('2024-01-23 15:30:00')

In [56]:
df_list = []
df_list.append(news_resp)
df_list.append(news_resp)

combined = pd.concat(df_list)
combined

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [63]:
news_path = Path.joinpath(root_dir, 'data', 'raw', 'boeing_news_20231101_to_20240131.csv')
news_df = pd.DataFrame(pd.read_csv(news_path))
display(news_df)



,Unnamed: 0,category,datetime,headline,id,image,related,source,summary,url
0,0,company,1706743226,UPDATE 1-US FAA says Boeing 737 audit will rev...,125468439,https://s.yimg.com/cv/apiv2/social/images/yaho...,BA,Yahoo,The Federal Aviation Administration said on We...,https://finnhub.io/api/news?id=3c9f0ede6c5e041...
1,1,company,1706741163,"ROSEN, TRUSTED INVESTOR COUNSEL, Encourages Th...",125443555,NaN,BA,Finnhub,"NEW YORK, Jan. 31, 2024 /PRNewswire/ -- WHY: ...",https://finnhub.io/api/news?id=ca83085b0a7e26b...
2,2,company,1706740216,Boeing (BA) Q4 2023 Earnings Call Transcript,125468440,https://g.foolcdn.com/editorial/images/1/featu...,BA,Yahoo,BA earnings call for the period ending Decembe...,https://finnhub.io/api/news?id=48410f9fbdcd990...
3,3,company,1706738780,US FAA says Boeing 737 audit will review Spiri...,125468441,https://media.zenfs.com/en/reuters-finance.com...,BA,Yahoo,WASHINGTON (Reuters) -The Federal Aviation Adm...,https://finnhub.io/api/news?id=9b6ab2295db1269...
4,4,company,1706738746,Boeing CEO says company has 'much to prove',125468442,https://s.yimg.com/ny/api/res/1.2/oFjcSDkfl1nj...,BA,Yahoo,Boeing chief executive Dave Calhoun told emplo...,https://finnhub.io/api/news?id=fe1736dabb96447...
...,...,...,...,...,...,...,...,...,...,...
208,208,company,1706514420,European Airline Ryanair's Q3 Profit Slumps As...,125385770,NaN,BA,Benzinga,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=4808ba7cd9ad5bb...
209,209,company,1706513760,Alaska Airlines plane said to have left Boeing...,125384252,NaN,BA,Seeking Alpha,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=cae0cb6eed1d96a...
210,210,company,1706512579,News Highlights : Top Company News of the Day ...,125373537,NaN,BA,Finnhub,Evergrande Was Once China's Biggest Property D...,https://finnhub.io/api/news?id=8d715bff70ed539...
211,211,company,1706512440,Ryanair narrows profit guidance following clas...,125388237,https://images.mktw.net/im-88479800/social,BA,MarketWatch,Various online travel agents stopped listing R...,https://finnhub.io/api/news?id=762cc1080f80ec4...


In [44]:

# news_df.to_csv('boeing_news_20231101_to_20240131.csv')

In [54]:
# General information
print(news_df.describe(include='all'))


NameError: name 'news_df' is not defined

In [71]:
# Check NA values
print(news_df.isna().sum())

# Convert UNIX to pandas datetime
news_df['datetime2'] = pd.to_datetime(news_df['datetime'], unit="s")
news_df.loc[pd.isna(news_df['summary']), :]


Unnamed: 0      0
category        0
datetime        0
headline        0
id              0
image         124
related         0
source          0
summary         2
url             0
datetime2       0
dtype: int64


,Unnamed: 0,category,datetime,headline,id,image,related,source,summary,url,datetime2
28,28,company,1706712966,The Boeing Company (BA) Q4 2023 Earnings Call ...,125437719,NaN,BA,SeekingAlpha,NaN,https://finnhub.io/api/news?id=97b3ae56c6a4ba0...,2024-01-31 14:56:06
143,143,company,1706586439,Boeing Customer Gol Files For Bankruptcy,125397543,NaN,BA,SeekingAlpha,NaN,https://finnhub.io/api/news?id=7f639fa386d7272...,2024-01-30 03:47:19


In [79]:
news_df2 = news_df.drop(columns=['Unnamed: 0', 'image'])
news_df3 = news_df2.dropna()
news_df3

,category,datetime,headline,id,related,source,summary,url,datetime2
0,company,1706743226,UPDATE 1-US FAA says Boeing 737 audit will rev...,125468439,BA,Yahoo,The Federal Aviation Administration said on We...,https://finnhub.io/api/news?id=3c9f0ede6c5e041...,2024-01-31 23:20:26
1,company,1706741163,"ROSEN, TRUSTED INVESTOR COUNSEL, Encourages Th...",125443555,BA,Finnhub,"NEW YORK, Jan. 31, 2024 /PRNewswire/ -- WHY: ...",https://finnhub.io/api/news?id=ca83085b0a7e26b...,2024-01-31 22:46:03
2,company,1706740216,Boeing (BA) Q4 2023 Earnings Call Transcript,125468440,BA,Yahoo,BA earnings call for the period ending Decembe...,https://finnhub.io/api/news?id=48410f9fbdcd990...,2024-01-31 22:30:16
3,company,1706738780,US FAA says Boeing 737 audit will review Spiri...,125468441,BA,Yahoo,WASHINGTON (Reuters) -The Federal Aviation Adm...,https://finnhub.io/api/news?id=9b6ab2295db1269...,2024-01-31 22:06:20
4,company,1706738746,Boeing CEO says company has 'much to prove',125468442,BA,Yahoo,Boeing chief executive Dave Calhoun told emplo...,https://finnhub.io/api/news?id=fe1736dabb96447...,2024-01-31 22:05:46
...,...,...,...,...,...,...,...,...,...
208,company,1706514420,European Airline Ryanair's Q3 Profit Slumps As...,125385770,BA,Benzinga,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=4808ba7cd9ad5bb...,2024-01-29 07:47:00
209,company,1706513760,Alaska Airlines plane said to have left Boeing...,125384252,BA,Seeking Alpha,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=cae0cb6eed1d96a...,2024-01-29 07:36:00
210,company,1706512579,News Highlights : Top Company News of the Day ...,125373537,BA,Finnhub,Evergrande Was Once China's Biggest Property D...,https://finnhub.io/api/news?id=8d715bff70ed539...,2024-01-29 07:16:19
211,company,1706512440,Ryanair narrows profit guidance following clas...,125388237,BA,MarketWatch,Various online travel agents stopped listing R...,https://finnhub.io/api/news?id=762cc1080f80ec4...,2024-01-29 07:14:00


In [80]:
# Double confirm there is no more na
print(news_df3.isna().sum())


category     0
datetime     0
headline     0
id           0
related      0
source       0
summary      0
url          0
datetime2    0
dtype: int64


In [82]:
# Check if there are any news between these two dates
tmp_df = news_df3[(news_df3['datetime2'] >= pd.to_datetime('2024-01-30')) & (news_df3['datetime2'] <= pd.to_datetime('2024-01-31'))]
print(len(tmp_df))


55


In [84]:
# Number of duplicated values in id
print(news_df['id'].duplicated().sum())

# Print unique values in different columns
for column in ['category', 'source', 'related']:
    print(f"Unique values in '{column}':")
    print(news_df[column].unique())

0
Unique values in 'category':
['company']
Unique values in 'source':
['Yahoo' 'Finnhub' 'MarketWatch' 'SeekingAlpha' 'Alliance News' 'TipRanks'
 'Benzinga' 'Seeking Alpha' 'Thefly.com' 'TalkMarkets' '247WallSt'
 'InvestorPlace' 'United Press International' 'PR Newswire'
 'Market News Video' 'Associated Press, The' 'GuruFocus']
Unique values in 'related':
['BA']


# Parse

In [138]:
# Simple fetch news from one url
def fetch_news_content(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Get the news content in <p> tag
        news_content = '\n'.join(p.get_text() for p in soup.find_all('p'))
        return news_content
    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return None

# More robust way to fetch news for a df
# Need to think of how to "add in" the content for rows with problem (additional iterations)
def fetch_news_content(df):
    # Create an empty list to store news content
    content_list = []

    cur_dir = Path.cwd()
    tar_dir = Path.joinpath(path_yq.get_root_dir(cur_dir=cur_dir), 'data', 'proc')
    for index, row in df.iterrows():
        try:
            if pd.isna(row['news_content']) or row['news_content'] in ['Timeout', 'Error', '']:
                response = requests.get(row['url'], timeout=5)
                soup = BeautifulSoup(response.content, 'html.parser')
                news_content = '\n'.join(p.get_text() for p in soup.find_all('p'))
                content_list.append(news_content)
            else:
                content_list.append(row['news_content'])
        except requests.exceptions.Timeout:
            print(f"Timeout occurred for index, url: {index}, {row['url']}")
            content_list.append("Timeout")
        except Exception as e:
            print(f"Error fetching content from {row['url']}: {e}")
            content_list.append("Error")
        
        # Optionally, save after every fetch or after a set number of fetches
        if index % 10 == 0:  # For example, save every 10 articles
            df.loc[:index, 'news_content'] = pd.Series(content_list)
            partial_file_path = Path.joinpath(tar_dir, 'boeing_partial_news_content_20231101_to_20240131.csv')
            df.to_csv(partial_file_path, index=False)
        
        # Implement delay to avoid being blocked by the server
        time.sleep(1)
    
    # print(content_list)
    # Update the DataFrame with all the fetched content
    df['news_content'] = pd.Series(content_list, index=df.index)
    # print(df)

    full_file_path = Path.joinpath(tar_dir, 'boeing_news_content_20231101_to_20240131.csv')
    # Save the final DataFrame to CSV
    df.to_csv(full_file_path, index=False)


In [139]:
# Assuming it has a url column
# fetch_news_content(news_df3)

# Try to fill up NA (the df is defined later)
# fetch_news_content(news_df5)

### Testing the fetch_news_content function

In [ ]:
#  Testing to fetch one line from news_df3
idx = 10
start_time = time.time()
print(news_df3.loc[idx, 'url'])
news_df4 = news_df3.loc[[idx]]
news_df4['news_content'] = news_df4['url'].apply(fetch_news_content)
end_time = time.time()
min, sec = divmod(end_time - start_time, 60)
print(f"The elapsed time for 1 url is {int(min)} minutes, {sec:.2f} seconds")
print(news_df4.loc[idx, 'news_content'])

# Clean News Content

## Check if the News Content are Successfully Fetched
- Some are blank after I try to run the url
- About 94 problematic rows

In [201]:
# Read the csv with complete news content from urls, and 
cur_dir = Path.cwd()
root_dir = path_yq.get_root_dir(cur_dir)
news_path = Path.joinpath(root_dir, 'data', 'proc', 'boeing_news_content_20231101_to_20240131.csv')
news_df5 = pd.DataFrame(pd.read_csv(news_path))



In [141]:
# Check if there is any unsuccessful events
error_df = news_df5[news_df5['news_content'].isin(['Timeout', 'Error', '']) | news_df5['news_content'].isna()]

if len(error_df):
    display(error_df)
else:
    print("All content are successfully fetched.")

,category,datetime,headline,id,related,source,summary,url,datetime2,news_content
16,company,1706718660,Wall Street Lunch: Powell Shuts The Door On Ma...,125440375,BA,SeekingAlpha,Stocks sink as Fed chief Jay Powell calls a Ma...,https://finnhub.io/api/news?id=118c521574a1747...,2024-01-31 16:31:00,NaN
19,company,1706717760,NEW YORK MARKET CLOSE: Dow knocked as Fed says...,125454152,BA,Alliance News,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=994a8d6eddd72e3...,2024-01-31 16:16:00,NaN
34,company,1706704020,UBS Sticks to Its Buy Rating for Boeing (BA),125468108,BA,TipRanks,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=3b2a6771d18d884...,2024-01-31 12:27:00,NaN
36,company,1706703060,Boeing Likely To Report Narrower Q4 Loss; Thes...,125431220,BA,Benzinga,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=10d8bd04b992aa8...,2024-01-31 12:11:00,NaN
37,company,1706702580,Boeing Earnings Preview: Watch The '25-26 Reve...,125431221,BA,Seeking Alpha,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=60c4def6860a419...,2024-01-31 12:03:00,NaN
...,...,...,...,...,...,...,...,...,...,...
204,company,1706518320,Jefferies aerospace/defense analyst to hold an...,125378132,BA,Thefly.com,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=5d50e14ef067c01...,2024-01-29 08:52:00,NaN
205,company,1706516460,Morning Brew: SoFi Surges on Q4 Earnings Beat ...,125381142,BA,GuruFocus,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=4c4901f5ffdc245...,2024-01-29 08:21:00,NaN
206,company,1706514420,European Airline Ryanair's Q3 Profit Slumps As...,125385770,BA,Benzinga,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=4808ba7cd9ad5bb...,2024-01-29 07:47:00,NaN
207,company,1706513760,Alaska Airlines plane said to have left Boeing...,125384252,BA,Seeking Alpha,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=cae0cb6eed1d96a...,2024-01-29 07:36:00,NaN


In [204]:
# Drop all the news_content with na
print(news_df5.isna().sum())
news_df6 = news_df5.dropna()
news_df6.reset_index(inplace=True, drop=True)
news_df6

# news_df6['url'].duplicated().sum()


category         0
datetime         0
headline         0
id               0
related          0
source           0
summary          0
url              0
datetime2        0
news_content    94
dtype: int64


,category,datetime,headline,id,related,source,summary,url,datetime2,news_content
0,company,1706743226,UPDATE 1-US FAA says Boeing 737 audit will rev...,125468439,BA,Yahoo,The Federal Aviation Administration said on We...,https://finnhub.io/api/news?id=3c9f0ede6c5e041...,2024-01-31 23:20:26,"(Adds background, comments, more from FAA)\nBy..."
1,company,1706741163,"ROSEN, TRUSTED INVESTOR COUNSEL, Encourages Th...",125443555,BA,Finnhub,"NEW YORK, Jan. 31, 2024 /PRNewswire/ -- WHY: ...",https://finnhub.io/api/news?id=ca83085b0a7e26b...,2024-01-31 22:46:03,\nPublished: 2024-01-31 22:46:03 ET\nBA\n\nNEW...
2,company,1706740216,Boeing (BA) Q4 2023 Earnings Call Transcript,125468440,BA,Yahoo,BA earnings call for the period ending Decembe...,https://finnhub.io/api/news?id=48410f9fbdcd990...,2024-01-31 22:30:16,"Founded in 1993, The Motley Fool is a financia..."
3,company,1706738780,US FAA says Boeing 737 audit will review Spiri...,125468441,BA,Yahoo,WASHINGTON (Reuters) -The Federal Aviation Adm...,https://finnhub.io/api/news?id=9b6ab2295db1269...,2024-01-31 22:06:20,By David Shepardson\nWASHINGTON (Reuters) -The...
4,company,1706738746,Boeing CEO says company has 'much to prove',125468442,BA,Yahoo,Boeing chief executive Dave Calhoun told emplo...,https://finnhub.io/api/news?id=fe1736dabb96447...,2024-01-31 22:05:46,Boeing President and CEO Dave Calhoun acknowle...
...,...,...,...,...,...,...,...,...,...,...
112,company,1706521290,"Trending tickers: Alphabet, Evergrande, Ryanai...",125373039,BA,Yahoo,The latest investor updates on stocks that are...,https://finnhub.io/api/news?id=957c0f571ba0cb8...,2024-01-29 09:41:30,What to expect from these mega-cap tech stars ...
113,company,1706519791,News Highlights : Top Company News of the Day ...,125375203,BA,Finnhub,"Amazon, iRobot Scrap Acquisition Deal Amazon.c...",https://finnhub.io/api/news?id=0b615f74fc33bdf...,2024-01-29 09:16:31,\nPublished: 2024-01-29 09:16:31 ET\nBA\n\nAma...
114,company,1706519724,Financiers grapple with plane shortages amid B...,125375204,BA,Finnhub,The financiers behind theworld's airline indus...,https://finnhub.io/api/news?id=bc5bf88b1afa7c8...,2024-01-29 09:15:24,\nPublished: 2024-01-29 09:15:24 ET\nBA\n\nDUB...
115,company,1706512579,News Highlights : Top Company News of the Day ...,125373537,BA,Finnhub,Evergrande Was Once China's Biggest Property D...,https://finnhub.io/api/news?id=8d715bff70ed539...,2024-01-29 07:16:19,\nPublished: 2024-01-29 07:16:19 ET\nBA\n\nThe...


In [213]:
# pd.to_datetime(1706519791 - 1706512579, unit='s')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example using 'news_content' column
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(news_df6['summary'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Define a threshold for identifying similar articles (e.g., 0.5)
threshold = 0.6

# Find articles that are similar based on the threshold
similar_pairs = []
for i in range(cosine_sim.shape[0]):
    for j in range(i+1, cosine_sim.shape[0]):
        if cosine_sim[i, j] > threshold:
            similar_pairs.append((i, j))

# similar_pairs contains pairs of indices in your DataFrame that are similar

In [10]:
print(len(similar_pairs))
print(similar_pairs)

duplicated_idx = set()
for i, j in similar_pairs:
    # Published less than 1 day
    if abs(news_df6.loc[i, 'datetime'] - news_df6.loc[j, 'datetime']) < 86400:
        duplicated_idx.add(j)
        print(f"Published on the same day")
    print(news_df6.loc[i, 'summary'])
    print(news_df6.loc[j, 'summary'])
    print('\n')

print(type(duplicated_idx))
duplicated_list = sorted(list(duplicated_idx))
print(duplicated_list)

# 87 is not inside

NameError: name 'similar_pairs' is not defined

In [209]:
# Drop the rows identified to be similar
news_df7 = news_df6.drop(index=duplicated_list, axis=0).reset_index(drop=True)
print(f"Dropped {len(duplicated_list)} rows with similar summaries.")
news_df7

Dropped 11 rows with similar summaries.


,category,datetime,headline,id,related,source,summary,url,datetime2,news_content
0,company,1706743226,UPDATE 1-US FAA says Boeing 737 audit will rev...,125468439,BA,Yahoo,The Federal Aviation Administration said on We...,https://finnhub.io/api/news?id=3c9f0ede6c5e041...,2024-01-31 23:20:26,"(Adds background, comments, more from FAA)\nBy..."
1,company,1706741163,"ROSEN, TRUSTED INVESTOR COUNSEL, Encourages Th...",125443555,BA,Finnhub,"NEW YORK, Jan. 31, 2024 /PRNewswire/ -- WHY: ...",https://finnhub.io/api/news?id=ca83085b0a7e26b...,2024-01-31 22:46:03,\nPublished: 2024-01-31 22:46:03 ET\nBA\n\nNEW...
2,company,1706740216,Boeing (BA) Q4 2023 Earnings Call Transcript,125468440,BA,Yahoo,BA earnings call for the period ending Decembe...,https://finnhub.io/api/news?id=48410f9fbdcd990...,2024-01-31 22:30:16,"Founded in 1993, The Motley Fool is a financia..."
3,company,1706738746,Boeing CEO says company has 'much to prove',125468442,BA,Yahoo,Boeing chief executive Dave Calhoun told emplo...,https://finnhub.io/api/news?id=fe1736dabb96447...,2024-01-31 22:05:46,Boeing President and CEO Dave Calhoun acknowle...
4,company,1706730240,13 Best Aerospace Stocks To Buy On Sale,125437670,BA,Yahoo,"In this article, we discuss the 13 best aerosp...",https://finnhub.io/api/news?id=f2c838db1660739...,2024-01-31 19:44:00,"In this article, we discuss the 13 best aerosp..."
...,...,...,...,...,...,...,...,...,...,...
101,company,1706521290,"Trending tickers: Alphabet, Evergrande, Ryanai...",125373039,BA,Yahoo,The latest investor updates on stocks that are...,https://finnhub.io/api/news?id=957c0f571ba0cb8...,2024-01-29 09:41:30,What to expect from these mega-cap tech stars ...
102,company,1706519791,News Highlights : Top Company News of the Day ...,125375203,BA,Finnhub,"Amazon, iRobot Scrap Acquisition Deal Amazon.c...",https://finnhub.io/api/news?id=0b615f74fc33bdf...,2024-01-29 09:16:31,\nPublished: 2024-01-29 09:16:31 ET\nBA\n\nAma...
103,company,1706519724,Financiers grapple with plane shortages amid B...,125375204,BA,Finnhub,The financiers behind theworld's airline indus...,https://finnhub.io/api/news?id=bc5bf88b1afa7c8...,2024-01-29 09:15:24,\nPublished: 2024-01-29 09:15:24 ET\nBA\n\nDUB...
104,company,1706512579,News Highlights : Top Company News of the Day ...,125373537,BA,Finnhub,Evergrande Was Once China's Biggest Property D...,https://finnhub.io/api/news?id=8d715bff70ed539...,2024-01-29 07:16:19,\nPublished: 2024-01-29 07:16:19 ET\nBA\n\nThe...


In [50]:
pd.to_datetime(1706512440, unit="s")

Timestamp('2024-01-29 07:14:00')

In [9]:
# Removed duplicates
cleaned_news_path = Path.joinpath(root_dir, 'data', 'proc', 'boeing_dedup_20231101_to_20240131.csv')
news_df7.to_csv(path_or_buf=cleaned_news_path, index=False)

NameError: name 'news_df7' is not defined

# Cleaning News Content
- Need to do it separately for each type
- Different techniques have different outputs
- Need to name the outputs based on techniques

- Notes from prof
    - here is a sample use for English: sentic.net/api/en/KEY.py?text=senticnet+is+pretty+cool

    - input text does not require any special formatting so feel free to use spaces instead of '+' or '%20'

    - ampersand, hashtag, semicolons, and braces ('&', '#', ';', '{', '}'), however, are illegal characters

    - hence, they should be replaced with colons (':') or removed entirely in the preprocessing phase

    - please note that:

    - 1) API keys are case-sensitive

    - 2) API keys will be valid for about one month

    - 3) API keys are personal and confidential

    - do not share nor use them from different devices or IP addresses

    - or else they will get terminated earlier

    - the capacity limit for our server is 8000 characters 

    - so our recommendation is to cap your input at about 1000 words

    - if you need to process bigger texts, you will have to split them into smaller parts

    - this is also a good idea in case you want to perform a finer-grained analysis of your input

    - all APIs, in fact, are designed to give you an overall judgement about the whole input

    - for more details, split your text into paragraphs or sentences and feed them to the API one by one

In [33]:
def clean_text(text):
    # Remove sentences that include "reporting by" (case insensitive)
    text = re.sub(r'\n.*?reporting by.*?\n', '', text, flags=re.IGNORECASE)

    # Remove everything starting with "\nPublished:" until the next newline
    text = re.sub(r'\nPublished:.*?\n', '', text, flags=re.IGNORECASE)

    # Remove everything including and after "\n(END)"
    text = re.sub(r'\n\(END\).*', '', text, flags=re.IGNORECASE)

    # Remove all the links
    text = re.sub(r'https?://.*?(?=\s|$)', '', text)

    # Remove the line that starts with "\nRelated Quotes"
    text = re.sub(r'\nRelated Quotes.*?\n', '', text, flags=re.IGNORECASE)

    # Remove content starting with "©" until the next newline
    # This matches the "©" symbol followed by any characters (.*?), until a newline
    text = re.sub(r'©.*?\n', '', text, flags=re.IGNORECASE)

    # To remove the specific sentences directly
    text = re.sub(r'Thank you for your patience\.\s+Our engineers are working quickly to resolve the issue\.', '', text)
    
    # Remove lines with fewer than 5 characters
    text = re.sub(r'^.{1,4}$', '', text, flags=re.MULTILINE)

    # Remove illegal characters for Sentic APIs
    text = re.sub(r'[&#;{}]', '', text)

    # Split the text into paragraphs, remove leading and trailing white spaces, and filter out empty strings
    paragraphs = [paragraph.strip() for paragraph in text.split('\n') if paragraph.strip()]
    
    if paragraphs == []:
        return np.nan
    print(paragraphs)
    return paragraphs  # Join the paragraphs back into a single string separated by newlines


In [34]:
news_df7 = pd.read_csv(filepath_or_buffer=cleaned_news_path, index_col=False)

# display(news_df7)
# Trying to spot the pattern for unrelated content
# for i in range(len(news_df7)):
#     news_txt = news_df7.loc[i, 'news_content']
#     display(news_txt)

news_df7['cleaned_news'] = news_df7['news_content'].apply(clean_text)
char_limit = 8000

news_df7

print(news_df7.isna().sum())
news_df7.dropna(inplace=True)
# passages2

['(Adds background, comments, more from FAA)', 'By David Shepardson', "WASHINGTON, Jan 31 (Reuters) - The Federal Aviation Administration said on Wednesday that its previously announced audit of Boeing 737 MAX manufacturing will look at all elements of production at Boeing and fuselage production at Spirit AeroSystems'.", 'The FAA said a team of two dozen aviation safety inspectors will conduct the audit at the Boeing 737 facility in Renton, Washington, and at Spirit in Wichita, Kansas. The audit will also examine how Boeing transfers unfinished work from suppliers to its production lines.', 'The FAA announced the audit on Jan. 13 after the agency grounded 171 MAX 9 planes due to a mid-air cabin panel blowout on a new Alaska Airlines MAX 9. The FAA last week allowed the planes to fly again.', 'The FAA said it is conducting enhanced oversight of Boeing and "will regularly assess trends, corrective actions and the effects of any changes to the quality system." The FAA did not say how lon

# Sentiment analysis
- SenticNet
- TextBlob
- nltk
- BERT

TODO:
- Check whether need to unify the results
- Not included subjectivity yet
- Check char limit

In [35]:
# number of tweets pass through API


# Use SenticNet APIs to do sentiment analysis
def sentic_api(text, KEY):
    APIURL = 'https://sentic.net/api/' + 'en' + '/' + KEY + '.py?text='
    
    # label is the concept, polarity that is returned from API
    label = str(requests.get(APIURL + text).content)[2:-3]
    return label

def sentic_analyse_polarity(text_list):
    polarity_list = []
    for i in range(len(text_list)):
        polarity_cat = sentic_api(text_list[i], POLARITY_API_KEY) # Polarity category (positive, negative)
        intensity = sentic_api(text_list[i], INTENSITY_API_KEY)
        if polarity_cat == "POSITIVE":
            polarity_list.append(intensity)
        elif polarity_cat == "NEGATIVE":
            polarity_list.append(-intensity)
        elif polarity_cat == "NEUTRAL":
            polarity_list.append(0)
        else:
            print(f"Unknown polarity: {polarity_cat}")
        print(f"Subjectivity is: {sentic_api(text_list[i], SUBJECTIVITY_API_KEY)}")
        if i % 10:
            print(f"{i}/{len(text_list)} is done.")
    return(np.mean(polarity_list))



In [19]:
news_df7['news_pol_sentic'] = news_df7['cleaned_news'].apply(sentic_analyse_polarity)
# TODO: Subjectivity

KeyboardInterrupt: 

In [36]:
def blob_analyse(df):
    for index, row in df.iterrows():
        # Ensure text_list is actually a list; if 'cleaned_news' is a single string of text, you might need `text_list = [row['cleaned_news']]`
        text_list = row['cleaned_news']
        polarity_list = []
        subjectivity_list = []
        
        # Assuming text_list is iterable; adjust if 'cleaned_news' is a singular text item
        for txt in text_list:
            # print(txt)
            blob = TextBlob(txt)
            polarity = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity
            polarity_list.append(polarity)
            subjectivity_list.append(subjectivity)
            # print(polarity, subjectivity)
        
        # Handling the case where lists might be empty to avoid ZeroDivisionError
        avg_polarity = np.mean(polarity_list) if polarity_list else 0
        avg_subjectivity = np.mean(subjectivity_list) if subjectivity_list else 0
        
        # Correctly assign back to the DataFrame
        df.loc[index, 'news_pol_blob'] = avg_polarity
        df.loc[index, 'news_sub_blob'] = avg_subjectivity

        # print(f"Row {index} is completed.")
        
        # Uncomment these print statements if you want to see the output for debugging
        # print(polarity_list)
        # print(subjectivity_list)

In [37]:
blob_analyse(news_df7)

(Adds background, comments, more from FAA)
0.5 0.5
By David Shepardson
0.0 0.0
WASHINGTON, Jan 31 (Reuters) - The Federal Aviation Administration said on Wednesday that its previously announced audit of Boeing 737 MAX manufacturing will look at all elements of production at Boeing and fuselage production at Spirit AeroSystems'.
-0.16666666666666666 0.16666666666666666
The FAA said a team of two dozen aviation safety inspectors will conduct the audit at the Boeing 737 facility in Renton, Washington, and at Spirit in Wichita, Kansas. The audit will also examine how Boeing transfers unfinished work from suppliers to its production lines.
0.0 0.0
The FAA announced the audit on Jan. 13 after the agency grounded 171 MAX 9 planes due to a mid-air cabin panel blowout on a new Alaska Airlines MAX 9. The FAA last week allowed the planes to fly again.
0.2028409090909091 0.44905303030303034
The FAA said it is conducting enhanced oversight of Boeing and "will regularly assess trends, corrective act

In [40]:
print(np.min(news_df7['news_sub_blob']))

0.0


In [ ]:


# nltk.download('vader_lexicon')  # Download the VADER lexicon if not already downloaded

sentence = 'OCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.'

sid = SentimentIntensityAnalyzer()
sentiment_scores = sid.polarity_scores(sentence)

print(sentiment_scores)

# VADER compound score: https://stackoverflow.com/questions/40325980/how-is-the-vader-compound-polarity-score-calculated-in-python-nltk

{'neg': 0.239, 'neu': 0.761, 'pos': 0.0, 'compound': -0.6597}


In [ ]:
from transformers import pipeline

# Load the sentiment analysis pipeline with the chosen BERT model
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_analysis = pipeline("sentiment-analysis", model=model_name)

# Example sentence
sentence = 'OCTOBER 2018: A Lion Air MAX plane crashes in Indonesia, killing all 189 people on board.'

# Get sentiment prediction
result = sentiment_analysis(sentence)

# Print the result
print(result)

# label is 1 to 5 stars. 1 is very negative, 5 is very positive
# Score is the confidence, between 0 to 1

config.json: 100%|██████████| 953/953 [00:00<00:00, 1.35MB/s]
pytorch_model.bin: 100%|██████████| 669M/669M [00:11<00:00, 59.6MB/s] 
/opt/homebrew/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
tokenizer_config.json: 100%|██████████| 39.0/39.0 [00:00<00:00, 877B/s]
vocab.txt: 100%|██████████| 872k/872k [00:00<00:00, 1.54MB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 284kB/s]


[{'label': '1 star', 'score': 0.8441336154937744}]


In [41]:
# After performing sentiment
stm_path = root_dir.joinpath('data', 'proc', 'boeing_stm_20231101_to_20240131.csv')
news_df7.to_csv(path_or_buf=stm_path, index=False)